In [ ]:
import numpy as np

class Mancala:
    def __init__(self, holes=7, stones=7, board=None):
        self.n_holes = holes
        self.n_stones = stones
        self.north_store = self.n_holes
        self.south_store = self.n_holes * 2 + 1
        self.north = 'north'
        self.south = 'south'
        self.first_play = None

        self.reset()

        if board is not None:
            self.board = board
    
    # make a board of initial state
    def reset(self):
        self.board = np.full((self.n_holes + 1) * 2, self.n_stones)
        self.board[self.n_holes] = 0
        self.board[-1] = 0
        self.first_play = True

    def _post_step(self, side, last_pos):
        # if last position is in the player's scoring well and is not first play
        # instruct an extra move
        if self.is_store(side, last_pos) and not self.first_play:
            return f'{side}:continue'

        # if the last position is in an self empty hole while the according opponent's hole
        # has stones in it, take the stones from both holes
        if self.is_hole(side, last_pos) and self.board[last_pos] == 1 \
            and self.board[self.get_opponent_pos(last_pos)] != 0:
            scores_stored_pos = self.get_store(side)
            self.board[scores_stored_pos] = self.board[scores_stored_pos]+ self.board[last_pos] \
                + self.board[self.get_opponent_pos(last_pos)]
            self.board[last_pos] = 0
            self.board[self.get_opponent_pos(last_pos)] = 0


        # record first play
        if self.first_play:
            self.first_play = False

        # if the player has won, print out the message
        if self.has_over_half_stones(side):
            return f'{side}:has won'
        else:
            return self.get_opponent_side(side)

    # select a hole to move by player
    def step(self, side, hole):
        if not self.has_legal_move(side):
            return self.check_win()
        
        pos = self._get_board_pos(side, hole)
        stones = self.board[pos]
        self.board[pos] = 0
        pos += 1
        # record the position where last stone is placed
        last_pos = pos
        while stones > 0:
            if not self.is_opponent_store(side, pos):
                self.board[pos] += 1
                stones -= 1
                if stones == 0:
                    last_pos = pos
            # add the position
            # if position outside the board make it start from the beginning of the board by %
            pos = (pos + 1) % len(self.board)
        
        return self._post_step(side, last_pos)
        
    
    # return the position of player's scoring well
    def get_store(self, side):
        return self.south_store if side == self.south else self.north_store
            
    # swap the player after each normal round
    def get_opponent_side(self, side):
        return self.north if side == self.south else self.south
    
    # verify whether it is player's hole
    def is_hole(self, side, pos):
        return side == self.south and self.north_store < pos < self.south_store \
            or side == self.north and pos < self.south_store
    
    # get the according opponent hole position given the self hole position
    def get_opponent_pos(self, self_pos):
        assert self.is_hole(self.south, self_pos) or self.is_hole(self.north, self_pos)
        return 2 * self.n_holes - self_pos
        
    # verify whether is player's scoring well
    def is_store(self, side, pos):
        return side == self.south and pos == self.south_store \
            or side == self.north and pos == self.north_store
    
    # verify whether is opponent's scoring well
    def is_opponent_store(self, side, pos):
        return side == self.south and pos == self.north_store \
            or side == self.north and pos == self.south_store
    
    # get the position to move in the board array
    # assuming the side chosen is reasonable
    def _get_board_pos(self, side, hole):
        return hole - 1 if side == self.north else hole + self.n_holes
    
    # swap the side for north according to pie rule
    # assuming the swap instruction is reasonable
    def swap_side(self):
        self.board = np.roll(self.board, self.n_holes + 1)
        return "swapped:south"
        # board_copy = deepcopy(self.board)
        # for i in range(len(self.board)):
        #     board_copy[i] = self.board[(i+self.n_holes+1)%len(self.board)]
        # self.board = board_copy
    
    # check whether the player has won
    def has_over_half_stones(self, side):
        return self.board[self.get_store(side)] > 49

    def get_start_hole(self, side):
        return 0 if side == self.north else self.n_holes + 1
    
    # check whether the player has legal move
    def has_legal_move(self, side):
        start = self.get_start_hole(side)
        return np.any(self.board[start:start + self.n_holes])
        # for i in range (self.n_holes):
        #     if self.board[i+offset] > 0:
        #         return True
        # return False

    # when one player does not have legal move
    # check the board to see who has won
    def check_win(self):
        north_total = self.board[self.get_store(self.north)]
        south_total = self.board[self.get_store(self.south)]
        offset = self.n_holes+1
        for i in range (self.n_holes):
            north_total += self.board[i]
            self.board[i] = 0
        self.board[self.get_store(self.north)] = north_total
        for i in range (self.n_holes):
            south_total += self.board[i+offset]
            self.board[i+offset] = 0
        self.board[self.get_store(self.south)] = south_total
        if north_total == south_total:
            return "draw"
        elif north_total > south_total:
            return "north:has won"
        else:
            return "south:has won" 
        
    # print out of the board
    def __str__(self):
        formatter = {'int': lambda x: f'{x: >3d}'}
        return '{:2d}  {}\n    {}  {}'.format(
            self.board[self.north_store],
            np.array2string(self.board[0:self.north_store][::-1], formatter=formatter),
            np.array2string(self.board[self.north_store+1:self.south_store], formatter=formatter),
            self.board[self.south_store]
        )

game = Mancala()
print(game)
print(game.step('south', 1))
print(game)
print(game.swap_side())

print(game)
print(game.board)
print(game.step('south', 1))
print(game.board)
print(game)
print(game.board)
print(game.step('south', 2))
print(game)
print(game.step('north', 2))
print(game)

print(game.step('south', 2))
print(game)


print(game.step('north', 1))
print(game)


print(game.step('south', 1))
print(game)
print(game.board)


print(game.step('north', 7))
print(game)
print(game.board)


print(game.step('south',3))
print(game)
print(game.step('north',7))
print(game)
print(game.step('north',5))
print(game)
print(game.step('south',7))
print(game)
print(game.step('north',7))
print(game)
print(game.step('north',6))
print(game)
print(game.step('north',7))
print(game)
print(game.step('north',4))
print(game)
print(game.step('south',6))
print(game)
print(game.step('north',7))
print(game)
print(game.step('north',6))
print(game)
print(game.step('south',5))
print(game)
print(game.step('north',7))
print(game)
print(game.step('north',5))
print(game)
print(game.step('south',7))
print(game)
print(game.step('north',6))
print(game)
print(game.step('north',7))
print(game)

# # one ending
# # south 不讲武德
# print(game.step('north',3))
# print(game)
# print(game.step('south',6))
# print(game)
# print(game.step('south',7))
# print(game)
# print(game.step('south',6))
# print(game)
# print(game.step('south',5))
# print(game)
# print(game.step('south',3))
# print(game)
# print(game.step('south',2))
# print(game)
# print(game.step('north',5))
# print(game)
# print(game.step('north',4))
# print(game)
# print(game.step('north',5))
# print(game)
# print(game.step('south',1))
# print(game)

# print(game.step('north',7))
# print(game)
# print(game.board)


# print(game.step('north',3))
# print(game)
# print(game.board)



# another ending
# 标准结局
print(game.step('north',4))
print(game)
print(game.step('south',4))
print(game)
print(game.step('north',7))
print(game)
print(game.step('north',3))
print(game)
print(game.step('south',7))
print(game)
print(game.step('north',7))
print(game)
print(game.step('north',6))
print(game)
print(game.board)
print(game.step('north',7))
print(game)
print(game.board)
print(game.step('north',5))
print(game)
print(game.board)

In [ ]:
# test for the board
def boardTest(board,board_to_compare):
    return np.array_equal(board,board_to_compare)

# test for the msg
def msgTest(msg,msg_to_compare):
    return msg == msg_to_compare

# test for the game board and output msg
def gameTest(msg,msg_to_compare,game,board_to_compare):
    return boardTest(game.board,board_to_compare) and msgTest(msg, msg_to_compare)




game = Mancala()

# Test the start of the board
print("Start of the game board is correct: ", end = " ")
print(boardTest(game.board,np.array([7,7,7,7,7,7,7,0,7,7,7,7,7,7,7,0])))

# Test a start move and the pie rule for no extra move
print("The game performs correctly for a start move: ", end = " ")
print(gameTest(game.step('south', 1), "north", game, np.array([7,7,7,7,7,7,7,0,0,8,8,8,8,8,8,1])))

# Test the swap case of North
print("The game performs correctly for a swap move: ", end = " ")
print(gameTest(game.swap_side(), "swapped:south", game, np.array([0,8,8,8,8,8,8,1,7,7,7,7,7,7,7,0])))

# Test a normal move
print("The game performs correctly for a normal move: ", end = " ")
print(gameTest(game.step('south', 2), "north", game, np.array([1,8,8,8,8,8,8,1,7,0,8,8,8,8,8,1])))

# Test a move which will lead to an extra move
game.board = np.array([0,8,8,8,8,8,8,1,7,7,7,7,7,7,7,0])
print("The game performs correctly for the extra move case: ", end = " ")
print(gameTest(game.step('south', 1), "south:continue", game, np.array([0,8,8,8,8,8,8,1,0,8,8,8,8,8,8,1])))


# Test a move which will take the stones from opponent
game.board = np.array([1,0,9,9,9,9,9,2 ,1,0,11,9,9,9,9,2])
print("The game performs correctly when need to take over opponet's stones: ", end = " ")
print(gameTest(game.step('north', 1), "south", game, np.array([0,0,9,9,9,9,9,12,1,0,11,9,9,0,9,2])))

# Test a move which passees two scoring wells
game.board = np.array([0,0,9,9,9,0,9,12,0,0,11,9,9,0,9,12])
print("The game performs correctly when passing both scoring wells: ", end = " ")
print(gameTest(game.step('north', 7), "south", game, np.array([0,0,9,9,9,0,0,24,1,1,12,10,10,1,0,12])))


# Test the case the game not end if one player has no legal move but it's his/her opponent's turn
game.board = np.array([0,0,0,0,0,0,0,42,0,0,1,16,2,2,1,34])
print("The game not end when the player has no legal move but is opponent's turn: ", end = " ")
print(gameTest(game.step('south', 3), "north", game, np.array([0,0,0,0,0,0,0,42,0,0,0,17,2,2,1,34])))

# Test the case the game ends if it's one player's turn and he/she has no legal move
game.board = np.array([0,0,0,0,0,0,0,42,0,0,1,16,2,2,1,34])
print("The game ends when it's a player's turn and no legal move left: ", end = " ")
print(gameTest(game.step('north', 3), "south:has won", game, np.array([0,0,0,0,0,0,0,42,0,0,0,0,0,0,0,56])))


# Test the case the game ends when both player has 49 stones
game.board = np.array([0,0,0,0,0,0,0,49,0,0,0,0,0,0,0,49])
print("The game ends with a draw when both players have 49 stones: ", end = " ")
print(gameTest(game.step('south', 7), "draw", game, np.array([0,0,0,0,0,0,0,49,0,0,0,0,0,0,0,49])))

# Test the case that the game not end when one player has 49 stones
game.board = np.array([7,4,0,2,2,0,1,48,2,6,1,1,2,3,0,19])
print("The game not end when one player has 49 stones: ", end = " ")
print(gameTest(game.step('north', 7), "north:continue", game, np.array([7,4,0,2,2,0,0,49,2,6,1,1,2,3,0,19])))

# Test the case that the game ends when one player has over 49 stones
print("The game ends when one player has over 49 stones: ", end = " ")
print(gameTest(game.step('north', 5), "north:has won", game, np.array([7,4,0,2,0,1,0,52,0,6,1,1,2,3,0,19])))

In [ ]:
class Parser:
    def __init__(self, server_text):
        self.msg = server_text
    
    
    ###  helper functions change an array of string to an array to number
    def str_to_num(self, string):
        return [int(str) for str in string]
    
    
    ###  helper function: turn an array of numbers into a board
    def msg_to_board(self, msg):
        board = self.str_to_num(msg)
        return '{:2d}  {}\n    {}  {}'.format(
            board[7],
            board[0:7][::-1],
            board[8:15],
            board[15]
        )
    
    ### helper function: create a initial board of 7,7
    def ini_board(self):
        ini_board = np.full(16, 7)
        ini_board[7] = 0
        ini_board[15] = 0
        str_board = self.msg_to_board(ini_board)
        return str_board, ini_board
    
    
    ######### end of helper functions #########
    def get_board(self):
#         print(self.msg)
        next_move = 'YOU'
        global str_board
        global board
        global moveswap
        result_msg = 'There is no swap.'
        if self.is_start():
            str_board, board = self.ini_board()
            next_move = 'YOU'
#             print(next_move)
        elif self.is_state_change():
            change_str, moveswap, board, side = self.msg.split(';')
            if moveswap.isdigit():                                         # move state
                print(next_move, ' moved ', moveswap, 'th hole.')
            else:                                                          # swap state
                print('The board has been swapped.')
                result_msg = 'The board has been swapped.'
            
#             print(result_msg)
            next_move = side
            print('The board after the state change is:')
            str_board = self.msg_to_board(board.split(','))
            board = self.str_to_num(board.split(','))
        return str_board, board, result_msg
    
    
    ### public functions ###
    def winner(self, board):
        n_score = board[7]
        s_score = board[-1]
        text = 'no winner'
        if n_score > s_score:
            text = 'You lose!'
        elif n_score == s_score:
            text = 'It is a draw!'
        else:
            text = 'You win!'
        return text
    
    def is_state_change(self):
        return self.msg.startswith('CHANGE') or self.msg.endswith('YOU') or self.msg.endswith('OPP')
    
    def is_start(self):
        return self.msg.startswith('START')
    
    def is_our_turn(self):
        return self.is_start() or self.msg.endswith('YOU')
    
    def is_end(self):
        return self.msg == 'END'
    
    def is_over(self):
        return self.msg.endswith('END')
    
    def not_recognizable(self):
        return not self.is_start() and not self.is_state_change() and not self.is_end() 
    
    def __str__(self):
        return self.msg
    

In [ ]:
class ParserTest:
    
    def __init__(self):
        self.text = ''      # text from server
        self.lose = 'You lose!'   #  result of lose
        self.draw = 'It is a draw!'    # result of draw
        self.win = 'You win!'   # result of win
        self.swap = 'The board has been swapped.'   # result of swapping
        self.no_swap = 'There is no swap.'  # result of no swapping
    
    def start_test(self):
        text = 'START:SOUTH'
        msg = Parser(text)
        assert not msg.is_state_change() and msg.is_our_turn() and not msg.is_over() and not msg.is_end()
        assert not msg.not_recognizable()
        assert msg.is_start()
        assert str(msg) == text
    
    def move_test(self):
        text = 'CHANGE;3;8,8,7,7,7,7,7,0,7,7,0,8,8,8,8,1;OPP'
        msg = Parser(text)
        print_board, board, result_msg = msg.get_board()
        assert not msg.is_start() and not msg.is_our_turn() and not msg.is_over() and not msg.is_end()
        assert result_msg == self.no_swap and not msg.not_recognizable()
        assert msg.is_state_change() and msg.winner(board) == self.win
        assert str(msg) == text
    
    def swap_test(self):
        text = 'CHANGE;SWAP;8,8,8,8,0,7,7,1,8,8,7,7,7,7,7,0;YOU'
        msg = Parser(text)
        print_board, board, result_msg = msg.get_board()
        assert not msg.is_start() and not msg.is_over() and not msg.is_end() and not msg.not_recognizable()
        assert msg.is_state_change() and msg.winner(board) == self.lose and msg.is_our_turn()
        assert result_msg == self.swap
        assert str(msg) == text
    
    def over_win_test(self):
        text = 'CHANGE;1;0,0,0,0,0,0,0,25,0,0,0,0,0,0,0,73;END'
        msg = Parser(text)
        print_board, board, result_msg = msg.get_board()
        assert not msg.is_start() and not msg.is_end() and not msg.not_recognizable()
        assert msg.is_over() and msg.is_state_change() and msg.winner(board) == self.win and not msg.is_our_turn()
        assert result_msg == self.no_swap
        assert str(msg) == text
        
    def over_lose_test(self):
        text = 'CHANGE;1;0,0,0,0,0,0,0,73,0,0,0,0,0,0,0,25;END'
        msg = Parser(text)
        print_board, board, result_msg = msg.get_board()
        assert not msg.is_start() and not msg.is_end() and not msg.not_recognizable()
        assert msg.is_over() and msg.is_state_change() and msg.winner(board) == self.lose and not msg.is_our_turn()
        assert result_msg == self.no_swap
        assert str(msg) == text
    
    def end_test(self):
        text = 'END'
        msg = Parser(text)
        assert msg.is_end() and msg.is_over()
        assert not msg.is_start() and not msg.is_state_change() and not msg.not_recognizable() and not msg.is_our_turn()
        assert str(msg) == text
    
    def wrong_message_test(self):
        text = 'aaaaabbbbb'
        msg = Parser(text)
        assert msg.not_recognizable() and not msg.is_start() and not msg.is_state_change() and not msg.is_our_turn()
        assert not msg.is_end() and not msg.is_over()
        assert str(msg) == text
        
    def runall(self):
        for obj in dir(ParserTest()):
            if obj.endswith('test'):
                getattr(ParserTest(), obj)()  # run all tests
    
test = ParserTest()
test.runall()

In [ ]:
import socket, random

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

print('connecting to localhost 12345')
sock.bind(('localhost', 12345))
sock.listen(1)
server, address = sock.accept()

def receive_line(client):
    buffer = bytearray()
    while True:
        data = client.recv(4)
        buffer.extend(data)
        if b'\n' in data:
            break
    return buffer

def send(msg):
    if str(msg).isdigit():                       # MOVE case
        text = 'MOVE;' + str(msg) + '\n'
        server.sendall(text.encode('utf-8'))
#         print('Send: ' + str(text))
    else:                                       # SWAP case
        text = 'SWAP\n'
        server.sendall(text.encode('utf-8'))
#         print('Send: ' + str(text))

def make_non_zero_move(board):
    move = random.randrange(7) + 1
#     print(move)
#     print(board)
#     print(board[7+move])
    while board[7+move] == 0:
#         print('The move on ', move, 'th hole is on an empty hole, try again')
        move = random.randrange(7) + 1
    send(move)

while True:
    text = receive_line(server).decode('utf-8').rstrip()
    msg = Parser(text)
    print(msg)
    # print the board whenever there is a message from the server
    if not msg.is_end():
        print_board, board, result_msg = msg.get_board()
        print(print_board)
        print()
        
        if msg.is_over():
            text = msg.winner(board)
            print(text, '\n', 'END')
            break
    
    if msg.is_our_turn():
        print('Your Turn! Make a move!')
        make_non_zero_move(board)

    if msg.not_recognizable():
        print('The error message is ', msg)
        print('Message not recognizable, game is over.')
        break